In [72]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [67]:
# Load in data that informs the times a customer could enter the store.
# TIME IS NOT IN DATETIME FORMAT!!!
time_probs = pd.read_csv('../cust_times.csv')
time_probs['time'] = pd.to_datetime(time_probs['time'])
time_probs.set_index('time', inplace=True)

In [376]:
time_probs

,new_id,probability
time,,
2023-03-01 06:59:00,10,0.001343
2023-03-01 07:00:00,6,0.000806
2023-03-01 07:01:00,4,0.000537
2023-03-01 07:02:00,12,0.001612
2023-03-01 07:03:00,10,0.001343
...,...,...
2023-03-01 21:45:00,9,0.001209
2023-03-01 21:46:00,5,0.000672
2023-03-01 21:47:00,9,0.001209


In [68]:
# Load in data that informs our functions which probability to use.
cust_probs = pd.read_csv('../probabilities.csv', index_col=0)
cust_probs

,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.103344,0.737061,0.058500,0.049789,0.051307
drinks,0.215688,0.010898,0.598540,0.087900,0.086974
entrance,0.000000,0.287576,0.153526,0.377435,0.181464
fruit,0.201447,0.095848,0.054804,0.597264,0.050637
spices,0.150565,0.193061,0.162979,0.090880,0.402515


In [3]:
# Create a list of possible states that does not include entrance 
# because no customer can return to the entrance.
states = cust_probs.drop('entrance').index
states

Index(['checkout', 'dairy', 'drinks', 'fruit', 'spices'], dtype='object', name='location')

In [5]:
# Testing how loc works. In the class, 'dairy' is taken out and 
# self.state takes its place.
cust_probs.loc['dairy']

checkout    0.103344
dairy       0.737061
drinks      0.058500
fruit       0.049789
spices      0.051307
Name: dairy, dtype: float64

In [375]:
class Customer:
    """
    A single customer that moves 
    through the supermarket in a MCMC simulation
    """
    
    def __init__(self, name, state='entrance', probs = cust_probs, times = time_probs):
        self.name = name
        self.state = state
        self.probs = probs
        self.times = times

    def __repr__(self):
        return f'<Customer {self.name} in {self.state}>'
    
    def next_state(self):
        """
        Simulates one customer's entire visit to the store.
        """
        visited = [self.state]
        start_time = np.random.choice(time_probs.index, p=time_probs.probability)
        close_time = '21:51:00'
        times = [start_time]
        
        while self.state != 'checkout':

            next_state = np.random.choice(states, p=self.probs.loc[self.state])

            start_time += pd.Timedelta('1 minute')

            if str(start_time)[11:] == close_time:
                self.state = 'checkout'
                visited.append('checkout')
                times.append(start_time)
                
            else:
                visited.append(next_state)
                times.append(start_time)
                self.state = next_state   
        
        df_times = pd.DataFrame(times)
        df_visited = pd.DataFrame(visited)

        df_combined = pd.merge(df_times, 
                               df_visited,
                               left_index=True, 
                               right_index=True
                               )
        df_combined.columns= ['time','location']
        df_combined['id'] = self.name
        
        return df_combined


class Supermarket:
    """
    Simulates an entire day of a store.
    """

    def __init__(self, no_customers) -> None:
        self.no_customers = no_customers

    def sim_day(self):

        cust_list = range(1, self.no_customers+1)

        df_full = pd.DataFrame()

        for cust in cust_list:
            customer_name = f'{cust}'
            customer_obj = Customer(customer_name)
            temp_df = customer_obj.next_state()
            df_full = pd.concat([df_full, temp_df])
        
        df_full.sort_values('time', inplace=True)
        df_full.set_index('time', inplace=True)

        return df_full


In [373]:
cust1 = Customer('Mykola')

In [374]:
cust1.next_state()

,time,location,id
0,2023-03-01 07:56:00,entrance,Mykola
1,2023-03-01 07:57:00,dairy,Mykola
2,2023-03-01 07:58:00,dairy,Mykola
3,2023-03-01 07:59:00,dairy,Mykola
4,2023-03-01 08:00:00,checkout,Mykola


In [371]:
lidl = Supermarket(1_500)

In [372]:
lidl.sim_day()

,location,id
time,,
2023-03-01 06:59:00,entrance,1159
2023-03-01 06:59:00,entrance,27
2023-03-01 07:00:00,dairy,1159
2023-03-01 07:00:00,spices,27
2023-03-01 07:00:00,entrance,1322
...,...,...
2023-03-01 21:51:00,checkout,862
2023-03-01 21:51:00,checkout,1140
2023-03-01 21:51:00,checkout,618
